### How to simulate a pathway simple promoter (+rbs?) + enzyme with kinetics using Tellurium and Antinomy

A model template has been initially generated using COPASI, although we will convert it into Antinomy model language in order to define modularly the model.

In [1]:
import tellurium as te
import numpy as np

Load the SBML template into Tellurium and simulate

In [115]:
pw = te.loadSBMLModel('prom_enzyme.xml')

In [117]:
results = []
for k in range(1, 2):
    pw.reset()
    s = pw.simulate(0, 3600, 1000)
    results.append(s)
    pw.plot(s, show=False, alpha=0.7)
te.show()

In [118]:
# Convert into Antinonmy
print(pw.getAntimony())

Exception: Antimony: 3 SBML error(s) when creating module 'New_Model'.  libAntimony tries to catch these errors before libSBML complains, but sometimes cannot.  Error message(s) from libSBML:

Outside of a <functionDefinition>, if a <ci> element is the first element within a MathML <apply>, then the <ci>'s value can only be chosen from the set of identifiers of <functionDefinition>s defined in the SBML model.
Reference: L3V1 Section 4.3.2
 The formula 'function(Catalysis_kcat, Enzyme, Substrate, Catalysis_Km)' in the math element of the <kineticLaw> uses 'function' which is not a function definition id.

The interpretation the Hierarchical Model Composition constructs to produce a kind of 'flattened' version of the model devoid of the comp package constructs must produce a valid SBML Level 3 model.
Reference: L3V1 Comp V1 Appendix A1
 Errors that follow relate to the flattened document produced using the CompFlatteningConverter.

Outside of a <functionDefinition>, if a <ci> element is 

Define a model definition that will connect three genes and promoters

In [5]:
def modelTemplate(nsteps):
    antinom = """
    // Created by libAntimony v2.9.4
        function Constant_flux__irreversible(v)
          v;
        end

        function Henri_Michaelis_Menten__irreversible(substrate, Km, V)
          V*substrate/(Km + substrate);
        end

        function Hill_Cooperativity(substrate, Shalve, V, h)
          V*(substrate/Shalve)^h/(1 + (substrate/Shalve)^h);
        end


        model New_Model()

          // Compartments and Species:
          compartment Cell;
          species Substrate in Cell, Product in Cell, Enzyme in Cell;
          species Inducer in Cell, Activated_promoter in Cell;

          // Reactions:
          Induction: Inducer => Activated_promoter; Cell*Hill_Cooperativity(Inducer, Induction_Shalve, Induction_V, Induction_h);
          Expression: Activated_promoter => Enzyme; Cell*Expression_k1*Activated_promoter;
          Leakage:  => Enzyme; Cell*Constant_flux__irreversible(Leakage_v);
          Degradation: Enzyme => ; Cell*Degradation_k1*Enzyme;
          Catalysis: Substrate => Product; Cell*Henri_Michaelis_Menten__irreversible(Substrate, Catalysis_Km, Catalysis_V);

         // Species initializations:
          Substrate = 0.5;
          Product = 0;
          Enzyme = 0;
          Inducer = 1;
          Activated_promoter = 0;

          // Compartment initializations:
          Cell = 1;

          // Variable initializations:
          Induction_Shalve = 0.01;
          Induction_V = 10000;
          Induction_h = 4;
          Expression_k1 = 100;
          Leakage_v = 0.0001;
          Degradation_k1 = 10;
          Catalysis_Km = 0.1;
          Catalysis_V = 0.1;

          // Other declarations:
          const Cell;

        end
        
        """
    antinom += "model *Big_Model()"+"\n"
    for i in np.arange(nsteps):
            antinom += "\t"+"m%d: New_Model();" % (i+1,)
            antinom += "\n"
    for i in np.arange(nsteps-1):
            antinom += "\t"+"m%d.Product is m%d.Substrate;" % (i+1, i+2)
            antinom += "\n"
    antinom += "end\n"
    return te.loada(antinom)

In [6]:
mod = modelTemplate(5)
mod['m3_Degradation_k1'] = 0.1
mod['m2_Degradation_k1'] = 0.1
mod['m5_Degradation_k1'] = 0.01
mod['m4_Catalysis_V'] = 0.1
mod['m5_Catalysis_Km'] = 1
for step in np.arange(2,6):
    mod['m'+str(step)+'_Substrate'] = 0

In [10]:
results = []
for k in range(1, 2):
    mod.reset()
    s = mod.simulate(0, 100, 100)
    results.append(s)
    mod.plot(s, show=False, alpha=0.7, loc='upperleft', figsize=(12,6))
te.show()

In [113]:
from pathSim import *

In [120]:
mp = metPath(2)
results = []
for k in range(1, 100):
    mp.sample()
    mp.model.reset()
    s = mp.model.simulate(0, 100, 100)
    results.append(s)
    mod.plot(s, show=False, alpha=0.7, loc='upperleft', figsize=(12,6))
te.show()


In [66]:
mp.model.getCompartmentVolumes()

array([1., 1., 1., 1.])

In [53]:
mp.model['m1_Copy_number']

10.0

In [80]:
mp.model.getFloatingSpeciesAmounts()

    m1_Substrate, m2_Substrate,   m1_Enzyme, m1_Inducer, m1_Activated_promoter, m2_Product,   m2_Enzyme,  m2_Inducer, m2_Activated_promoter
 [[   8.2283e-16,  1.30103e-10, 5.76653e-08, 0.00597405,           2.19571e-08,          1, 7.27906e-08, 0.000190888,           1.21681e-09]]